# Springleaf Marketing Response

## Features Creation and Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import collections

In [2]:
dtypeSpecificator = {'VAR_0008':object,'VAR_0009':object,'VAR_0010':object,
                     'VAR_0011':object,'VAR_0012':object,'VAR_0043':object,
                     'VAR_0157':object,'VAR_0167':object,'VAR_0177':object,
                     'VAR_0196':object,'VAR_0214':object,'VAR_0226':object,
                     'VAR_0229':object,'VAR_0230':object,'VAR_0232':object,
                     'VAR_0236':object,'VAR_0239':object }

dataTrain = pd.read_csv("Data/train.csv", dtype = dtypeSpecificator)
dataTest = pd.read_csv("Data/test.csv", dtype = dtypeSpecificator)
x_data = dataTrain.drop(['ID','target'],axis=1)
x_data_t = dataTest.drop(['ID'],axis=1)
y_train = dataTrain['target']
x_data.fillna(-1,inplace=True)
x_data_t.fillna(-1,inplace=True)
dataTrain.head(5)

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,...,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0,0,false,false,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1,0,false,false,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0,0,false,false,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0,0,false,false,...,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0,0,false,false,...,98,98,998,999999998,998,998,9998,9998,BRANCH,1


### Raw Data Cleaning
- Find type of columns

In [49]:
#Data type extraction

# Find type for each columns.
int_data = []
float_data = []
object_data = []

for var in x_data.columns.values:
    if x_data[var].dtype == 'object':
        object_data.append(var)
    elif  x_data[var].dtype == 'int64':
        int_data.append(var)
    elif x_data[var].dtype == 'float64':
        float_data.append(var)
            

# Find number of unique values for each columns.
out_data = []
low_data = []
low2_data = []
high_data = []
in_out_data = []

for var in x_data.columns.values:
    if len(x_data[var].unique()) == 1:
        out_data.append(var)
    elif len(x_data[var].unique()) == 2:
        in_out_data.append(var)
    elif  len(x_data[var].unique()) < 20:
        low_data.append(var)
    elif len(x_data[var].unique()) < 80:
        low2_data.append(var)
    else:
        high_data.append(var)

In [4]:
#remove irrelevant data
x_data.drop(out_data,axis=1,inplace=True)

duplicate = {}
dups = []

#remove duplicates
for sub_data in [in_out_data,low_data,low2_data,high_data]:
    for i in range(len(sub_data)):
        var1 = sub_data[i]
        if var1 not in dups:
            duplicate[var1] = []
            for j in range(len(sub_data) - i - 1):
                var2 =  sub_data[i+j+1]
                if var2 not in dups:
                    if len(x_data[var1].unique()) == len(x_data[var2].unique()):
                        if  (x_data[var1] == x_data[var2]).all():
                            duplicate[var1].append(var2)
                            dups.append(var2)
                            
x_data.drop(dups,axis=1,inplace=True)
x_data_t.drop(dups,axis=1,inplace=True)
print len(dups)

Examine Object columns data to treat them

In [6]:
print "number of object data : ", len(object_data)
print "number of int data : ", len(int_data)
print "number of float data : ", len(float_data)
print "number of columns : ", x_data.shape[1]

date_col = ["VAR_0073","VAR_0075","VAR_0156","VAR_0157","VAR_0158","VAR_0159","VAR_0166","VAR_0167","VAR_0168","VAR_0169","VAR_0176","VAR_0177","VAR_0178","VAR_0179","VAR_0204","VAR_0217"]

city_name = ["VAR_0200"]

state_name = ["VAR_0237","VAR_0274"]

employement_data = ["VAR_0404", "VAR_0493"]

year_col = ["VAR_0294","VAR_0314","VAR_0332","VAR_0531"]

#for var in high_data[100:120]:
#    if var not in object_data :
#        k = len(x_data[var].unique())*10
#        print var, k/10
#        plt.plot(x_data[y_train == 0][var].value_counts())
#        plt.plot(x_data[y_train == 1][var].value_counts())
#        plt.show()

number of object data :  51
number of int data :  1404
number of float data :  477
number of columns :  1878


### Treat Data 

First convert date_col, year_col into time stamp

In [7]:
# Treat date
import time 

dateFormat = "%d%b%y:%H:%M:%S"
yearFormat = "%Y"
yearmonthFormat = "%Y%m"

for var in date_col:
    x_data[var] =  x_data[var].apply(lambda x : time.mktime(time.strptime(x, dateFormat)) if x != -1 else x)
    x_data_t[var] =  x_data_t[var].apply(lambda x : time.mktime(time.strptime(x, dateFormat)) if x != -1 else x)
    
for var in year_col[:-1]:
    x_data[var] = x_data[var].apply(lambda x : time.mktime(time.strptime(str(int(x)), yearFormat)) if x != -1 else x)
    x_data_t[var] = x_data_t[var].apply(lambda x : time.mktime(time.strptime(str(int(x)), yearFormat)) if x != -1 else x)
    
for var in year_col[-1:]:
    x_data[var] = x_data[var].apply(lambda x : time.mktime(time.strptime(str(int(x)), yearmonthFormat)) if x != -1 else x)
    x_data_t[var] = x_data_t[var].apply(lambda x : time.mktime(time.strptime(str(int(x)), yearmonthFormat)) if x != -1 else x)

Convert employement data using CountVectorizer

In [29]:
#Treat employement data


stopwords ='and of a or in on'.split(' ')

from sklearn.feature_extraction.text import CountVectorizer

counts = []
dfs = []
dfst = []
for var in employement_data:
    count = CountVectorizer(min_df=50,binary=True,stop_words=stopwords)
    x_data[var] = x_data[var].apply(lambda x: "undetermined" if x == -1 else x)
    x_data_t[var] = x_data_t[var].apply(lambda x: "undetermined" if x == -1 else x)
    test = count.fit_transform(x_data[var])
    test2 = count.transform(x_data_t[var])
    counts.append(count)
    dfs.append(test)
    dfst.append(test2)
dfs

[<145231x43 sparse matrix of type '<type 'numpy.int64'>'
 	with 16817 stored elements in Compressed Sparse Row format>,
 <145231x63 sparse matrix of type '<type 'numpy.int64'>'
 	with 18268 stored elements in Compressed Sparse Row format>]

Convert city data using CountVectorizer

In [36]:
#treat city data

from sklearn.feature_extraction.text import CountVectorizer

countsc = []
dfsc = []
dfsct = [] 
for var in city_name:
    count = CountVectorizer(min_df=200,binary=True)
    x_data[var] = x_data[var].apply(lambda x: "undetermined" if x == -1 else x.replace(' ','_'))
    x_data_t[var] = x_data_t[var].apply(lambda x: "undetermined" if x == -1 else x.replace(' ','_'))
    test = count.fit_transform(x_data[var])
    test2 = count.transform(x_data_t[var])
    countsc.append(count)
    dfsc.append(test)
    dfsct.append(test2)
dfsc

[<145231x96 sparse matrix of type '<type 'numpy.int64'>'
 	with 38283 stored elements in Compressed Sparse Row format>]

Use sklearn to shatter categorical data

In [10]:
categorical_data = []
for var in object_data:
    if var not in date_col + city_name + employement_data + dups:
        categorical_data.append(var)

########## eclatement
from sklearn.feature_extraction import DictVectorizer as DV
dat_dict = x_data[categorical_data].T.to_dict().values()
dat_dict2 = x_data_t[categorical_data].T.to_dict().values()
vectorizer = DV(sparse = False)
vectorizer.fit(dat_dict)
dat = vectorizer.transform(dat_dict)
dat2 = vectorizer.transform(dat_dict2)
x_cat = pd.DataFrame(dat)
x_cat_t = pd.DataFrame(dat2)
print x_cat.shape

#x_cat2 = x_data[categorical_data].copy()
#
########## creation de label dans une colonne unique
#from sklearn.preprocessing import LabelEncoder
#for var in categorical_data:
#    le = LabelEncoder()
#    le.fit(x_data[var])
#    x_cat2[var] = le.transform(x_data[var])
#
#print x_cat2.shape

(145231, 252)


### Create New Features from existing columns
time related columns

In [20]:
time_f = []

time_f.append('uncompleted_date')

x_data['uncompleted_date'] = x_data[date_col].apply(lambda x: len(x[x==-1])*1.0/len(date_col),axis=1)
x_data_t['uncompleted_date'] = x_data_t[date_col].apply(lambda x: len(x[x==-1])*1.0/len(date_col),axis=1)

for i in range(len(date_col)):
    for j in range(i+1,len(date_col)):
        var_f = 'delta_time_'+str(i)+'_'+str(j)
        time_f.append(var_f)
        x_data[var_f] = x_data[date_col[j]] - x_data[date_col[i]] 
        x_data_t[var_f] = x_data_t[date_col[j]] - x_data_t[date_col[i]] 
        
print len(time_f)

['uncompleted_date', 'delta_time_0_1', 'delta_time_0_2', 'delta_time_0_3', 'delta_time_0_4', 'delta_time_0_5', 'delta_time_0_6', 'delta_time_0_7', 'delta_time_0_8', 'delta_time_0_9', 'delta_time_0_10', 'delta_time_0_11', 'delta_time_0_12', 'delta_time_0_13', 'delta_time_0_14', 'delta_time_0_15', 'delta_time_1_2', 'delta_time_1_3', 'delta_time_1_4', 'delta_time_1_5', 'delta_time_1_6', 'delta_time_1_7', 'delta_time_1_8', 'delta_time_1_9', 'delta_time_1_10', 'delta_time_1_11', 'delta_time_1_12', 'delta_time_1_13', 'delta_time_1_14', 'delta_time_1_15', 'delta_time_2_3', 'delta_time_2_4', 'delta_time_2_5', 'delta_time_2_6', 'delta_time_2_7', 'delta_time_2_8', 'delta_time_2_9', 'delta_time_2_10', 'delta_time_2_11', 'delta_time_2_12', 'delta_time_2_13', 'delta_time_2_14', 'delta_time_2_15', 'delta_time_3_4', 'delta_time_3_5', 'delta_time_3_6', 'delta_time_3_7', 'delta_time_3_8', 'delta_time_3_9', 'delta_time_3_10', 'delta_time_3_11', 'delta_time_3_12', 'delta_time_3_13', 'delta_time_3_14', 'd

is both state the same ?

In [40]:
x_data['varstate'] = x_data[state_name].apply(lambda x: 1 if x[state_name[0]] == x[state_name[1]] else 0,axis =1)
x_data_t['varstate'] = x_data_t[state_name].apply(lambda x: 1 if x[state_name[0]] == x[state_name[1]] else 0,axis =1)

Count error code to understand influence

In [44]:
#create some new indicators

max_value = {}

mod_col = list(set(int_data + float_data).difference(dups + out_data))

for col in mod_col:
    max_value[col] = x_data[col].max()
    
review_col = []
for col in mod_col:
    if (max_value[col] + 1) == (10**len(str(max_value[col]))) and len(str(max_value[col])) > 1:
        review_col.append(col)
review_col = np.array(review_col)

max_list = []
for col in review_col:
    max_list.append(max_value[col])
max_list = np.array(max_list)        
        
mod = ['count_99','count_98','count_97','count_96']

def count_errors(line):
    res = line.copy()
    vals = line[review_col].values
    for i in range(4):
        res[mod[i]] = max_list[vals == (max_list - i)].shape[0]
        #res[review_col[vals == (max_list - i)]] = -1
    return res
          
for col in mod: 
    x_data[col] = np.zeros(x_data.shape[0])
    x_data_t[col] = np.zeros(x_data_t.shape[0])

cuts = np.r_[np.arange(0,145231,10000),[145231]]
for i in range(15):
    x_data.ix[cuts[i]:cuts[i+1]-1,:] = x_data.ix[cuts[i]:cuts[i+1]-1,:].apply(lambda x : count_errors(x),axis=1)
    print i
    
cuts = np.r_[np.arange(0,145232,10000),[145232]]
for i in range(15):
    x_data_t.ix[cuts[i]:cuts[i+1]-1,:] = x_data_t.ix[cuts[i]:cuts[i+1]-1,:].apply(lambda x : count_errors(x),axis=1)
    print i

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


Merge the features and save them for the next step

In [45]:
cols = list(set(x_data.columns).difference(categorical_data + city_name + employement_data))

x_train = np.concatenate((x_data[cols],x_cat,dfs[0].todense(),dfs[1].todense(),dfsc[0].todense()),axis=1).astype('int32')
x_test = np.concatenate((x_data_t[cols],x_cat_t,dfst[0].todense(),dfst[1].todense(),dfsct[0].todense()),axis=1).astype('int32')

In [48]:
np.savetxt("Features/train1309",x_train)
np.savetxt("Features/ytrain1309",y_train)
np.savetxt("Features/test1309",x_test)